<a href="https://colab.research.google.com/github/lfpbarros/Data-Challenge-Coodesh-20221121/blob/main/Data_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Challenge - challenge by coodesh

### Carregando os arquivos

In [1]:
from google.colab import drive
drive.mount('/content/drive') # para facilitar a visibilidade do projeto, optei por fazer no Google Colab e com isso, salvei cópias dos dados no google drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
titulos_netflix = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Challenge TTT/data/netflix_titles.csv', sep=',', encoding='ISO-8859-1')
titulos_prime = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Challenge TTT/data/amazon_prime_titles.csv', sep=',', encoding='ISO-8859-1')

Importei os arquivos que salvei no drive e para evitar qualquer tipo de erro deixei claro qual era o separador e a forma como queria o encoding.

In [4]:
titulos_netflix['streaming'] = 'Netflix'
titulos_prime['streaming'] = 'Amazon Prime Video'

Sabendo que iria mesclar as bases e poderíamos ter ids duplicados, optei por criar uma coluna categórica em cada DataFrame informando a que streaming ele pertence. Isso vai ser útil em futuras filtragens.

## Mesclar as bases

In [5]:
titulos = pd.concat([titulos_netflix, titulos_prime])
titulos.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix


Para mesclar os dados, optei por simplesmente colar um em cima do outro como eles tem o mesmo tamanho de colunas, a minha ideia era mexer o mínimo possível com os dados antes das filtragens.

## Top 10 atores/atrizes por todos os dados

In [6]:
# ver o que precisa limpar da series

In [7]:
elenco = titulos['cast'].to_list()
lista_limpar = []
for i in elenco:
  lista = str(i).split(',')
  for ator in lista:
    if len(ator) < 4:
      lista_limpar.append(ator)

Ao estudar a base de dados, percebi que por vezes na coluna de cast, encontrava algumas impurezas nos dados. Daí, criei esse loop com o objetivo de criar uma lista com algumas delas, o meu interesse era pegar a coluna de elenco e ter o retorno de uma lista apenas com nome de atores.

In [8]:
set(lista_limpar) # utilizei o set para ter valores únicos

{' ',
 ' 2',
 ' 3',
 ' A',
 ' EL',
 ' JP',
 ' JR',
 ' LB',
 ' MD',
 ' TJ',
 '1',
 'BT',
 'DMX',
 'Dev',
 'GPS',
 'Jai',
 'K K',
 'Kii',
 'L',
 'LB',
 'Lea',
 'MAX',
 'MHD',
 'Mia',
 'Nas',
 'Non',
 'PNL',
 'Sri',
 'TC',
 'TJ',
 'Vir',
 'You',
 'nan'}

In [9]:
lista_limpar = [' ', ' 2',' 3',' A','1','L','nan']

Criei uma lista com o que considerei informações que são sujeiras do dataframe.

Meu critério para definir o que é sujeira: ser nan, numérico ou apenas uma letra, abreviações como LB resolvi manter por terem atores que as utilizam

In [10]:
def atores(cast):
  elenco = cast.to_list()
  lista_atores = []
  for i in elenco:
    if i != 'NaN':
      lista = str(i).split(',')
      for ator in lista:
        if ator not in lista_limpar:
          lista_atores.append(ator.strip())
  actors = pd.Series(lista_atores)
  return actors

Criei a função atores. Essa função pega uma variável do dataframe, tranforma em lista, faz o tratamento de dados e quando há vários itens separados por vírgula como no caso de elenco, ela os separa e os individualiza numa lista que é transformada em uma Series.

In [11]:
actors = atores(titulos['cast'])
actors.value_counts()[:10].to_frame().rename(columns={0:'count'})

,count
Anupam Kher,59
Maggie Binkley,56
Amitabh Bachchan,47
Shah Rukh Khan,46
Nassar,43
Akshay Kumar,41
Paresh Rawal,39
Naseeruddin Shah,39
Danny Trejo,39
Om Puri,35


Apliquei a função criada na variável 'cast' e utilizei o método value_counts para termos a contagem de atores, filtrei para exibir apenas o top 10

## Top 5 países produtores de conteúdos considerando todos os dados e comparando as duas plataformas

In [12]:
paises_top_5 = titulos['country'].value_counts()[:5].to_frame().rename(columns={'country':'total_filmes'})
paises_top_5

,total_filmes
United States,3071
India,1201
United Kingdom,447
Japan,247
South Korea,199


Criei a variável para ter o top 5 de países produtores de conteúdo, também usei o value_counts para isso.

In [13]:
netflix_paises = titulos_netflix['country'].value_counts().to_frame().rename(columns={'country':'netflix'})
prime_paises = titulos_prime['country'].value_counts().to_frame().rename(columns={'country':'prime'})

Para ter a comparação entre os países criei a contagem de valores por plataforma de streaming para fazer o merge com o top 5 que havia criado.

In [14]:
qtd_por_streaming = pd.merge(netflix_paises, prime_paises,how="left", left_index=True, right_index=True)
paises_top_5_por_streaming = pd.merge(paises_top_5, qtd_por_streaming, how='left', left_index=True, right_index=True).fillna(0)

In [15]:
paises_top_5_por_streaming['prime'] = paises_top_5_por_streaming['prime'].astype(int)
paises_top_5_por_streaming = paises_top_5_por_streaming[['netflix', 'prime', 'total_filmes']]
paises_top_5_por_streaming

,netflix,prime,total_filmes
United States,2818,253,3071
India,972,229,1201
United Kingdom,419,28,447
Japan,245,2,247
South Korea,199,0,199


Fiz o merge e reorganizei as colunas para ter o DataFrame com a quantidade por plataforma de streaming e o total de filmes por país

## Mês no qual há mais adições de filmes na plataforma Netflix

In [16]:
titulos_netflix['date_added'] = pd.to_datetime(titulos_netflix['date_added']) # converti a coluna de data de adições da netflix para datetime
titulos_netflix['month_year'] = titulos_netflix['date_added'].dt.to_period('M') # criei uma nova coluna apenas com o ano e mês das adições

In [17]:
titulos_netflix['month_year'].value_counts()[:1]

2021-07    257
Freq: M, Name: month_year, dtype: int64

In [18]:
print('O mês com mais adições de filmes na Netflix foi 07/2021 com 257 adições')

O mês com mais adições de filmes na Netflix foi 07/2021 com 257 adições


Usei o value_counts para agregar as categorias de mês e filtrei para exibir o primeiro lugar

## Quantidade de filmes listados como comédia

No DataFrame quando se menciona comédia, temos como gênero a palavra 'Comedy' ou a palavra 'Comedies'. Assim, me dei a liberdade de fazer duas análises. Na primeira a quantidade de filmes onde comédia é listada como um dos gêneros e outra onde apenas comédia é listada como gênero.

In [19]:
titulos_filmes = titulos.query('type == "Movie"')
titulos_filmes.listed_in.str.contains('Comed').value_counts()

False    10107
True      3838
Name: listed_in, dtype: int64

Criei um dataframe filtrando apenas filmes e usei o método contains para procurar em 'listed_in' qualquer filme que tivesse 'Comed', a escolha dessa string foi porque tanto 'Comedy' quanto 'Comedies' começam com 'Comed'

In [20]:
qtd_filmes_comedia = len(titulos_filmes[titulos_filmes.listed_in.str.contains('Comed')])

In [21]:
print(f'A quantidade de filmes que comédia é listada como um dos gêneros é de {qtd_filmes_comedia}')

A quantidade de filmes que comédia é listada como um dos gêneros é de 3838


In [22]:
titulos_comedia = titulos_filmes.query('listed_in == "Comedy" or listed_in == "Comedies"')
qtd_apenas_comedia = len(titulos_comedia)

In [23]:
print(f'A quantidade de filmes apenas listados como comédia é de {qtd_apenas_comedia}')

A quantidade de filmes apenas listados como comédia é de 552


Para ter os números apenas de comédia, criei uma variável para filtrar apenas comédia.

## Lista de todos os gêneros de filmes

In [24]:
lista_generos = set(titulos_filmes['listed_in'].to_list())
lista_generos_final = []
for i in lista_generos:
  lista = i.split(',')
  for j in lista:
    lista_generos_final.append(j.strip())

lista_generos_final = set(lista_generos_final)
print('Lista de gêneros de filmes do DataFrame:', end=' \n')
for i in lista_generos_final:
  print(i, end=', \n')

Lista de gêneros de filmes do DataFrame: 
Faith and Spirituality, 
Documentary, 
Thrillers, 
Special Interest, 
Independent Movies, 
International Movies, 
Music Videos and Concerts, 
Military and War, 
Entertainment, 
Faith & Spirituality, 
Drama, 
Arts, 
Romance, 
Horror, 
Fitness, 
Sports Movies, 
Horror Movies, 
Unscripted, 
Historical, 
Stand-Up Comedy, 
Fantasy, 
Action & Adventure, 
Adventure, 
Sports, 
Classic Movies, 
Comedies, 
Suspense, 
Arthouse, 
Dramas, 
Comedy, 
Movies, 
Talk Show and Variety, 
LGBTQ, 
Animation, 
Cult Movies, 
Sci-Fi & Fantasy, 
Action, 
Science Fiction, 
Young Adult Audience, 
Anime Features, 
LGBTQ Movies, 
Documentaries, 
Romantic Movies, 
International, 
and Culture, 
Children & Family Movies, 
Anime, 
Western, 
Kids, 
Music & Musicals, 


Peguei todos os gêneros do dataframe filtrado por filmes e em casos de mais de um gênero separei pela vírgula e usei o set para remover as duplicatas.

## A frequênica de "TV Show" de todos os dados e comparativamente em relação as duas plataformas

In [25]:
tv_show = titulos.query('type == "TV Show"')
tv_show_por_streaming = tv_show['streaming'].value_counts().to_frame().T
tv_show_por_streaming['total'] = tv_show['streaming'].value_counts().values.sum()

In [26]:
print(f'A frequência de TV Show no dataframe é de {len(tv_show)}')
tv_show_por_streaming

A frequência de TV Show no dataframe é de 4530


,Netflix,Amazon Prime Video,total
streaming,2676,1854,4530


Criei o dataframe filtrado por TV Show, utilizei a coluna streaming para ter a quantidade de tv shows por streaming e uma coluna com o total, daí temos o total e a comparação por serviço de streaming.

## A frequência de "Movies" de todos os dados e comparativamente em relação as duas plataformas

In [27]:
movies = titulos.query('type == "Movie"')
movies_por_streaming = movies['streaming'].value_counts().to_frame().T
movies_por_streaming['total'] = movies['streaming'].value_counts().values.sum()

In [28]:
print(f'A frequência de Movies no dataframe é de {len(movies)}')
movies_por_streaming

A frequência de Movies no dataframe é de 13945


,Amazon Prime Video,Netflix,total
streaming,7814,6131,13945


Criei o dataframe filtrado por Movies, utilizei a coluna streaming para ter a quantidade de filmes por streaming e uma coluna com o total, daí temos o total e a comparação por serviço de streaming.